In [433]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split

# 1. Explorando o Dataset

O Dataset escolhido foi do site kaggle: https://www.kaggle.com/datasets/khanghunhnguyntrng/football-players-transfer-fee-prediction-dataset



In [434]:
df = pd.read_csv("final_data.csv")

In [435]:
df.head()

,player,team,name,position,height,age,appearance,goals,assists,yellow cards,...,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,/david-de-gea/profil/spieler/59377,Manchester United,David de Gea,Goalkeeper,189.0,32.0,104,0.000000,0.000000,0.009585,...,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,/jack-butland/profil/spieler/128899,Manchester United,Jack Butland,Goalkeeper,196.0,30.0,15,0.000000,0.000000,0.069018,...,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0
2,/tom-heaton/profil/spieler/34130,Manchester United,Tom Heaton,Goalkeeper,188.0,37.0,4,0.000000,0.000000,0.000000,...,0.616438,0.924658,292,697,84,4,600000,6000000,1,0
3,/lisandro-martinez/profil/spieler/480762,Manchester United,Lisandro Martínez,Defender Centre-Back,175.0,25.0,82,0.028090,0.056180,0.224719,...,0.000000,0.000000,6408,175,22,9,50000000,50000000,2,0
4,/raphael-varane/profil/spieler/164770,Manchester United,Raphaël Varane,Defender Centre-Back,191.0,30.0,63,0.017889,0.017889,0.053667,...,0.000000,0.000000,5031,238,51,21,40000000,80000000,2,0


In [436]:
df.columns

Index(['player', 'team', 'name', 'position', 'height', 'age', 'appearance',
       'goals', 'assists', 'yellow cards', 'second yellow cards', 'red cards',
       'goals conceded', 'clean sheets', 'minutes played', 'days_injured',
       'games_injured', 'award', 'current_value', 'highest_value',
       'position_encoded', 'winger'],
      dtype='object')

In [437]:
df.shape
# Muitas linhas será interessante para predizer um valor de um jogador fictício

(10754, 22)

Para aprimorar a avaliação do modelo de previsão, as estatísticas, a saber, "gols", "assistências", "cartões amarelos", "segundos cartões amarelos", "cartões vermelhos", "gols sofridos" e "gols sem sofrer gols", foram transformadas para uma base por 90 minutos. Essa transformação envolveu a divisão de cada valor estatístico pelo valor correspondente por 90 minutos (calculado como minutos jogados divididos por 90).

É importante ressaltar que os dados são de 2 temporadas 2021-2022 e 2022-2023, então os valores serão diferentes de hoje em dia (2025).

De acordo com a análise das colunas do dataset, os melhores parâmetros para realizar uma predição do valor de mercado de um jogador são:
1. Time ✔
2. Posição
3. Altura ✔
4. Idade ✔
5. Aparições (em partidas) ✔
6. Gols ✔
7. Assistências ✔
8. Cartão Amarelo ✔
9. 2o Cartão Amarelo
10. Cartão Vermelho
11. Gols Concedidos ✔
12. "Clean Sheets" ✔
13. Minutos jogados ✔
14. Dias Machucado ✔
15. Partidas Machucado ✔
16. Troféus ✔
17. Valor atual (euro)

Posição encoded e outras colunas serão apenas para identificação e outras métricas.

As informações de Time poderão ser agrupadas em ligas, então a ideia será criar uma nova coluna representando a liga em que o jogador jogar, pois possivelmente por time ficaria muito complicado (?)

Temos meio campistas mais defensivos que darão menos assistências, mais parecidos como defensores, ao mesmo tempo que temos meio campistas mais atacantes, mais próximos de um atacante. Tal fato será explorado no dataset, e possivelmente iremos adicionar eles em diferentes PosiçõesEncoded baseado em sua Posição, fazendo esse tratamento antes de aplicar o modelo.

## Embasamento teórico do modelo preditivo:

Iremos utilizar os dados já obtidos para prever o valor de um jogador fictício, que esteja presente em algum desses clubes/ligas, dando a ele todas as características listadas anteriormente como parâmetros de entrada, e de saída iremos retornar o seu valor de mercado estimado. Algo interessante que podemos fazer também é remover um jogador arbitrariamente do dataset e testar o modelo nele, removendo seu valor de mercado, para avaliar se o modelo está se saindo bem em sua predição.

A princípio, iremos utilizar o seguinte raciocínio: Teremos os valores dos jogadores como pontos em um gráfico de N dimensões. Cada eixo irá corresponder a um parâmetro escolhido. A ideia será realizar a média das distâncias dos pontos mais próximos do ponto que queremos encontrar (no caso o valor do jogador fictício). A média da distância será o valor do jogador em questão. O nome desse modelo é o KNN simples, e irei explicar passo a passo de como ele funciona:

O modelo KNN simples é uma técnica que leva em consideração a proximidade entre os dados para a realização de predições. Dados similares tendem a estar concentrados na mesma região no espaço de dispersão dos dados. Uma boa forma de explicar esse modelo é pensar em um espaço 2D, onde temos o eixo x e y. Nesse espaço, já possuímos pontos marcados nele (que vem dos dados do dataset, mais especificamente do conjunto de validação) e vamos inserir um novo ponto, onde não temos seu valor. A partir dele, iremos calcular a distância desse nosso novo ponto com os outros pontos vizinhos dele. Essa distância é a distância euclidiana:

suponha $x_0 , y_0$ sendo as coordenadas do ponto que colocamos

- d = $\sqrt{(x_0 - x_1)² + (y_0 - y_1)²}$

Para um espaço de dimensão 3 teríamos:

- d = $\sqrt{(x_0 - x_1)² + (y_0 - y_1)² + (z_0 - z_1)²}$

Para um espaço de dimensão N teríamos:

- d = $\sqrt{(x_0 - x_1)² + (y_0 - y_1)² + (z_0 - z_1)² + ... + (n_0 - n_1)²}$

Lembrando que essas fórmulas são para visualização da distância entre o ponto que escolhemos, com um outro ponto já colocado no nosso espaço. No nosso modelo, iremos usar 3 vizinhos, logo, teríamos que realizar essa fórmula para esses 3 pontos. Algo como:

- d_total = $\sum_{i=1}^{j}\sqrt{(x_0 - x_i)² + (y_0 - y_i)² + (z_0 - z_i)² + ... + (n_0 - n_i)²}$

Sendo $j = 3$ pois estamos pensando em 3 vizinhos.

Com as distâncias calculadas, realizamos a média e chegamos em um resultado. Esse resultado é o valor que estamos prevendo, sendo então o valor de mercado estimado do nosso jogador.

Temos que ter cuidado com as escalas dos eixos, esse será um grande desafio nessa modelagem. Escolher a melhor forma de representar os dados tem um grande impacto no resultado final.

In [438]:
# Cada liga tem 20 times em um geral, depende da liga
liga_inglesa = df["team"].unique()[0:20]
liga_alema = df["team"].unique()[20:38]
liga_espanhola = df["team"].unique()[38:58]
liga_italiana = df["team"].unique()[58:78]
liga_francesa = df["team"].unique()[78:98]
liga_holandesa = df["team"].unique()[98:116]
liga_brasileira = df["team"].unique()[116:136]
liga_portuguesa = df["team"].unique()[136:154]
liga_mexicana = df["team"].unique()[154:172]
liga_russa = df["team"].unique()[172:188]
liga_2ainglesa = df["team"].unique()[188:212]
liga_turca = df["team"].unique()[212:231]
liga_austriaca = df["team"].unique()[231:243]
liga_estadounidense = df["team"].unique()[243:272]
liga_argentina = df["team"].unique()[272:300]
liga_japonesa = df["team"].unique()[300:318]
liga_arabe = df["team"].unique()[318:334]
liga_coreana = df["team"].unique()[334:346]
liga_sulafricana = df["team"].unique()[346:361]
liga_australiana = df["team"].unique()[361:374]

In [439]:
# Criando um mapeamento de times para ligas
liga_map = {
    **{team: 1 for team in liga_inglesa},
    **{team: 2 for team in liga_alema},
    **{team: 3 for team in liga_espanhola},
    **{team: 4 for team in liga_italiana},
    **{team: 5 for team in liga_francesa},
    **{team: 6 for team in liga_holandesa},
    **{team: 7 for team in liga_brasileira},
    **{team: 8 for team in liga_portuguesa},
    **{team: 9 for team in liga_mexicana},
    **{team: 10 for team in liga_russa},
    **{team: 11 for team in liga_2ainglesa},
    **{team: 12 for team in liga_turca},
    **{team: 13 for team in liga_austriaca},
    **{team: 14 for team in liga_estadounidense},
    **{team: 15 for team in liga_argentina},
    **{team: 16 for team in liga_japonesa},
    **{team: 17 for team in liga_arabe},
    **{team: 18 for team in liga_coreana},
    **{team: 19 for team in liga_sulafricana},
    **{team: 20 for team in liga_australiana},
}

# Adicionar uma nova coluna 'liga' ao dataframe
df['league'] = df['team'].map(liga_map)

# Verificar se há times sem mapeamento
times_sem_liga = df[df['league'].isnull()]['team'].unique()
if len(times_sem_liga) > 0:
    print("Times sem liga mapeada:", times_sem_liga)

# Preencher times sem liga com um valor padrão (opcional)
df['league'] = df['league'].fillna(0)

Removendo colunas que não serão utilizadas

In [440]:
# df = df.drop(["winger","player","highest_value", "name"],axis=1)

# Selecionar apenas as colunas numéricas relevantes
columns = [
    'league', 'height', 'age', 'appearance',
    'goals', 'assists', 'yellow cards', 'second yellow cards', 'red cards',
    'goals conceded', 'clean sheets', 'minutes played', 'days_injured',
    'games_injured', 'award', 'current_value', 'highest_value',
    'position_encoded', 'winger'
]
df = df[columns]
df

,league,height,age,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,1,189.000000,32.0,104,0.000000,0.000000,0.009585,0.0,0.000000,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,1,196.000000,30.0,15,0.000000,0.000000,0.069018,0.0,0.000000,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0
2,1,188.000000,37.0,4,0.000000,0.000000,0.000000,0.0,0.000000,0.616438,0.924658,292,697,84,4,600000,6000000,1,0
3,1,175.000000,25.0,82,0.028090,0.056180,0.224719,0.0,0.000000,0.000000,0.000000,6408,175,22,9,50000000,50000000,2,0
4,1,191.000000,30.0,63,0.017889,0.017889,0.053667,0.0,0.000000,0.000000,0.000000,5031,238,51,21,40000000,80000000,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10749,20,181.240353,20.0,16,0.175953,0.087977,0.263930,0.0,0.000000,0.000000,0.000000,1023,0,0,0,75000,75000,4,1
10750,20,190.000000,24.0,26,0.372671,0.186335,0.186335,0.0,0.000000,0.000000,0.000000,1449,102,18,0,300000,300000,4,0
10751,20,181.240353,19.0,20,0.375000,0.000000,0.187500,0.0,0.000000,0.000000,0.000000,960,0,0,0,50000,50000,4,0
10752,20,181.240353,20.0,17,0.312139,0.104046,0.000000,0.0,0.104046,0.000000,0.000000,865,0,0,0,50000,50000,4,0


In [441]:
df.head(4)

,league,height,age,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,1,189.0,32.0,104,0.00000,0.00000,0.009585,0.0,0.0,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,1,196.0,30.0,15,0.00000,0.00000,0.069018,0.0,0.0,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0
2,1,188.0,37.0,4,0.00000,0.00000,0.000000,0.0,0.0,0.616438,0.924658,292,697,84,4,600000,6000000,1,0
3,1,175.0,25.0,82,0.02809,0.05618,0.224719,0.0,0.0,0.000000,0.000000,6408,175,22,9,50000000,50000000,2,0


# TESTANDO TREINO TESTE E VALIDAÇÃO

- Treinamento: onde o modelo aprende.
- Validação: onde você ajusta hiperparâmetros (como profundidade da árvore, número de vizinhos, regularização etc).
- Teste: onde você avalia o desempenho final. Esse conjunto deve ficar intocado até o final.

In [442]:
# --- 1. Separar X e y
X = df[columns].drop(columns=["current_value"])
y = df["current_value"]

In [443]:
# --- 2. Separar treino+validação e teste
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [444]:
# --- 3. Separar treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)
# Resultado: 60% treino, 20% validação, 20% teste

In [445]:

# --- 4. Normalização feita apenas com o treino (importantíssimo!)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [446]:
# Previsão no conjunto de validação
k = 5
preds_val = []

for i in range(len(X_val_scaled)):
    distances = euclidean_distances(X_train_scaled, [X_val_scaled[i]])
    indices = np.argsort(distances.ravel())[:k]
    mean_value = y_train.iloc[indices].mean()
    preds_val.append(mean_value)

# (Opcional) Avaliar erro no conjunto de validação
from sklearn.metrics import mean_absolute_error
val_error = mean_absolute_error(y_val, preds_val)
print(f"Erro médio absoluto na validação: €{val_error:.2f}")


Erro médio absoluto na validação: €1795154.81


In [447]:
# Jogador fictício (ex: Lisandro Martínez)
jogador_ficticio = {
    "league": 1, "height": 175, "age": 25, "appearance": 82, "goals": 0.02809, 
    "assists": 0.05618, "yellow cards": 0.224719, "second yellow cards": 0.0, "red cards": 0.0,
    "goals conceded": 0, "clean sheets": 0, "minutes played": 6408, "days_injured": 175,
    "games_injured": 22, "award": 9, "highest_value": 50000000,
    "position_encoded": 2, "winger": 0
}

# Transformar e prever
jogador_ficticio_df = pd.DataFrame([jogador_ficticio])
normalized_jogador = scaler.transform(jogador_ficticio_df)

distances = euclidean_distances(X_train_scaled, normalized_jogador)
indices = np.argsort(distances.ravel())[:k]
predicted_value = y_train.iloc[indices].mean()

print(f"Valor predito para o jogador Lisandro Martínez: €{predicted_value:.2f}")


Valor predito para o jogador Lisandro Martínez: €49400000.00


In [448]:
euro = "€"

# Normalizar os dados numéricos
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df.drop(columns=["current_value"]))
normalized_data[0:2]

array([[0.00000000e+00, 6.60000000e-01, 6.07142857e-01, 9.71962617e-01,
        0.00000000e+00, 0.00000000e+00, 3.19488833e-04, 0.00000000e+00,
        0.00000000e+00, 1.35250266e-01, 3.72736954e-03, 9.87381703e-01,
        1.78799489e-02, 1.47492625e-02, 1.41304348e-01, 3.50000000e-01,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.00000000e-01, 5.35714286e-01, 1.40186916e-01,
        0.00000000e+00, 0.00000000e+00, 2.30061350e-03, 0.00000000e+00,
        0.00000000e+00, 1.38036810e-01, 2.30061350e-03, 1.37118822e-01,
        2.17113665e-01, 1.71091445e-01, 1.08695652e-02, 1.10000000e-01,
        0.00000000e+00, 0.00000000e+00]])

In [449]:
df.head(4)

,league,height,age,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,1,189.0,32.0,104,0.00000,0.00000,0.009585,0.0,0.0,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,1,196.0,30.0,15,0.00000,0.00000,0.069018,0.0,0.0,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0
2,1,188.0,37.0,4,0.00000,0.00000,0.000000,0.0,0.0,0.616438,0.924658,292,697,84,4,600000,6000000,1,0
3,1,175.0,25.0,82,0.02809,0.05618,0.224719,0.0,0.0,0.000000,0.000000,6408,175,22,9,50000000,50000000,2,0


### Realizando uma verificação

Utilizando um jogador do dataset para verificar como nosso algoritmo está funcionando
- Jogador: Lizandro Martinez (linha 3 do dataset)

In [450]:
target_values = df["current_value"].values

# Inserindo jogador fictício (no caso são os status de Lizandro Martinez)
jogador_ficticio = {
       "league": 1, "height": 175, "age": 25, "appearance": 82, "goals": 0.02809, 
       "assists": 0.05618, "yellow cards": 0.224719, "second yellow cards": 0.0, "red cards": 0.0,
       "goals conceded": 0, "clean sheets": 0, "minutes played": 6408, "days_injured": 175,
       "games_injured": 22, "award": 9, "highest_value": 50000000,
       "position_encoded": 2, "winger": 0
}

# jogador_ficticio = {
#     "league": 1, "height": 175, "age": 25, "appearance": 82, "goals": 0.02809, 
#     "assists": 0.05618, "yellow cards": 0.224719, "goals conceded": 0, "clean sheets": 0, 
#     "minutes played": 6408, "days_injured": 175, "games_injured": 22	, 
#     "award": 9
# }

jogador_ficticio_df = pd.DataFrame([jogador_ficticio])
normalized_jogador = scaler.transform(jogador_ficticio_df)
normalized_jogador

array([[0.        , 0.38      , 0.35714286, 0.76635514, 0.00249689,
        0.014045  , 0.00749063, 0.        , 0.        , 0.        ,
        0.        , 0.67381703, 0.07449979, 0.06489676, 0.09782609,
        0.25      , 0.33333333, 0.        ]])

Após termos os dados normalizados, podemos aplicar o KNN simples nesse dataset e obter o valor de mercado do jogador em questão

In [451]:
# Implementando KNN simples
distances = euclidean_distances(normalized_data, normalized_jogador)
k = 5  # Número de vizinhos ímpar
indices = np.argsort(distances.ravel())[:k]
predicted_value = target_values[indices].mean()

print(f"Valor predito para o jogador Lizandro Martinez: €{predicted_value:.2f}")

Valor predito para o jogador Lizandro Martinez: €51000000.00


OBS: eu testei antes sem a liga, e o valor do Lisandro Martinez dava 5milhoes de diferença do valor real, depois de adicionar a liga foi para 2milhoões de erro, o que é uma boa melhora. Depois de adicionar todas as colunas numéricas ficamos com 1milhão de diferenã, melhor ainda!

# Markdown das coisas tiradas

Além disso, temos que levar em conta que cada posição valoriza certos tipos de atributos mais do que outros. Um goleiro fazer gols não é muito comum, e durante a execução do modelo isso pode influenciar muito, o que não era pra acontecer. Teremos que aplicar certos pesos baseado na coluna de "position encoded", o que será essencial para uma predição mais precisa.

Lista de parâmetros que irão valer mais para cada PosiçãoEncoded:

1 - Goleiro:
 - Gols Concedidos (quanto menor, melhor)
 - "Clean Sheets"
 - Altura

2 - Defensor:
 - Cartão Amarelo (quanto menor, melhor)
 - 2o Cartão Amarelo (quanto menor, melhor)
 - Cartão Vermelho (quanto menor, melhor)
 - Altura

3 - Meio Campista:
 - Assistências
 - Cartão Vermelho (quanto menor, melhor)

4 - Atacante:
 - Gols
 - Assistências
